In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

In [ ]:
#Importing and pre-processing the images
DATADIR = "Datasets/PetImages"
CATEGORIES = ["Cat", "Dog"]

training_data = []
IMG_SIZE = 70

def create_training_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))#, cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                training_data.append([new_array, class_num])
            except Exception as e:
                pass

create_training_data()

In [ ]:
print(len(training_data))

In [ ]:
#Randomizing the data
import random

random.shuffle(training_data)

In [ ]:
X = []
y = []

In [ ]:
for features, label in training_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
y = np.array(y).reshape(-1,1)

In [ ]:
X = X/255.0 #Converting the numbers to float

In [ ]:
X_val = X[: 1000]   #Validation set
y_val = y[: 1000]

In [ ]:
index = int(X.shape[0]*0.2) #Deciding what % of data will be in the test set. Currently set to 20%
X_train, X_test = X[index:], X[:index]
y_train, y_test = y[index:], y[:index]

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
import time

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  # Invalid device or cannot modify virtual devices once initialized.
  pass

In [ ]:
# Trying out different combinations of layers
dense_layers = [1,2]
layer_sizes = [64, 128]
conv_layers = [1,2,3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            
            NAME = f'dense{dense_layer}-layer_size{layer_size}-conv_layer{conv_layer}-{int(time.time())}'
            tensorboard = TensorBoard(log_dir=f'logs/{NAME}')
            
            filepath=f"checkpoints/weights-improvement-d{dense_layer}-l{layer_size}-c{conv_layer}.hdf5"
            checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
            
            early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
            
            model = Sequential()

            model.add(Conv2D(layer_size, (3,3), activation="relu", input_shape = X_train.shape[1:]))
            model.add(MaxPooling2D(pool_size=(2,2), strides=2))

            for l in range(conv_layer - 1):
                model.add(Conv2D(layer_size, (3,3), activation="relu"))
                model.add(MaxPooling2D(pool_size=(2,2), strides=2))

            model.add(Flatten())

            for l in range(dense_layer):
                model.add(Dense(layer_size, activation= "relu"))
                model.add(Dropout(0.2))

            model.add(Dense(1, activation= "sigmoid"))

            model.compile(loss="binary_crossentropy", 
                            optimizer="adam", 
                            metrics=["accuracy"])

            model.fit(X_train, y_train, batch_size=32, 
                            epochs=25, 
                            validation_data=(X_test, y_test),
                            callbacks= [tensorboard, checkpoint, early_stop])
            
            model.load_weights(filepath)
            model.save(f'saved_models/cats_dogs_classifier-d{dense_layer}-l{layer_size}-c{conv_layer}.h5')

In [ ]:
#Best settings for good results
dense_layer = 1
layer_size = 64
conv_layer = 3

NAME = f'dense{dense_layer}-layer_size{layer_size}-conv_layer{conv_layer}-{int(time.time())}'
tensorboard = TensorBoard(log_dir=f'logs/{NAME}')
            
filepath=f"checkpoints/weights-improvement-d{dense_layer}-l{layer_size}-c{conv_layer}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
            
early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
            
model = Sequential()

model.add(Conv2D(layer_size, (3,3), activation="relu", input_shape = X_train.shape[1:]))
model.add(MaxPooling2D(pool_size=(2,2), strides=2))

model.add(Conv2D(layer_size, (3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2), strides=2))

model.add(Conv2D(layer_size, (3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2), strides=2))

model.add(Flatten())

model.add(Dense(layer_size, activation= "relu"))
model.add(Dropout(0.2))

model.add(Dense(1, activation= "sigmoid"))

model.compile(loss="binary_crossentropy", 
                optimizer="adam", 
                metrics=["accuracy"])

model.fit(X_train, y_train, batch_size=32, 
                epochs=25, 
                validation_data=(X_test, y_test),
                callbacks= [tensorboard, checkpoint, early_stop])

model.load_weights(filepath)
model.save(f'saved_models/cats_dogs_classifier-d{dense_layer}-l{layer_size}-c{conv_layer}.h5')

In [ ]:
model.load_weights('checkpoints/weights-improvement-d1-l64-c3.hdf5')

In [ ]:
y_predicted = model.predict(X_val)
y_predicted= (y_predicted>0.5)

#Tests to check if used model is good
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

print(f'Accuracy score is {accuracy_score(y_val, y_predicted)}')
print(f'\nClassification report:\n{classification_report(y_val, y_predicted)}')
print(f'\nConfusion matrix:\n{confusion_matrix(y_predicted, y_val)}')

In [ ]:
#Making custom predictions
def make_prediction(test_img):
    t_img = cv2.imread(test_img)#, cv2.IMREAD_GRAYSCALE)
    t_img_rz = cv2.resize(t_img, (IMG_SIZE, IMG_SIZE))
    t_img_rs = t_img_rz.reshape(-1, IMG_SIZE, IMG_SIZE, 3)
    result = model.predict(t_img_rs)
    return result

In [ ]:
pred = make_prediction('smile_cat.jpg')
print(CATEGORIES[int(pred[0][0])])